In [15]:
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

# Import Data And Take Initial Look

In [10]:
raw_dataset = pd.read_csv('data/dataset.csv', sep=";")
raw_dataset.head()

,uuid,default,account_amount_added_12_24m,account_days_in_dc_12_24m,account_days_in_rem_12_24m,account_days_in_term_12_24m,account_incoming_debt_vs_paid_0_24m,account_status,account_worst_status_0_3m,account_worst_status_12_24m,...,status_3rd_last_archived_0_24m,status_max_archived_0_6_months,status_max_archived_0_12_months,status_max_archived_0_24_months,recovery_debt,sum_capital_paid_account_0_12m,sum_capital_paid_account_12_24m,sum_paid_inv_0_12m,time_hours,worst_status_active_inv
0,63f69b2c-8b1c-4740-b78d-52ed9a4515ac,0.0,0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,...,1,1,1,1,0,0,0,178839,9.653333,1.0
1,0e961183-8c15-4470-9a5e-07a1bd207661,0.0,0,0.0,0.0,0.0,NaN,1.0,1.0,1.0,...,1,1,2,2,0,0,0,49014,13.181389,NaN
2,d8edaae6-4368-44e0-941e-8328f203e64e,0.0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,1,1,2,2,0,0,0,124839,11.561944,1.0
3,0095dfb6-a886-4e2a-b056-15ef45fdb0ef,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,1,1,0,0,0,324676,15.751111,1.0
4,c8f8b835-5647-4506-bf15-49105d8af30b,0.0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,0,1,1,1,0,0,0,7100,12.698611,NaN


In [11]:
raw_dataset.shape

(99976, 43)

We have roughly 100,000 total rows of data.

In [43]:
len(set(raw_dataset.uuid))

99976

Each row represents a different uuid, so we can use this as our index down the line.

In [12]:
sum(raw_dataset.default.fillna(0))

1288.0

Of those 100,000 rows, 1,288 are defaults. <br>
This means we have an imbalanced dataset (so we'll use SMOTE to balance it out).<br>

In [13]:
len(raw_dataset[raw_dataset['default'].isnull()].index.tolist())

10000

10% of this dataset is unlabeled. So let's separate out these indices to know what we're training on v. predicting on.

In [16]:
raw_dataset.isnull().sum()/raw_dataset.shape[0]

uuid                                   0.000000
default                                0.100024
account_amount_added_12_24m            0.000000
account_days_in_dc_12_24m              0.118388
account_days_in_rem_12_24m             0.118388
account_days_in_term_12_24m            0.118388
account_incoming_debt_vs_paid_0_24m    0.593292
account_status                         0.543861
account_worst_status_0_3m              0.543861
account_worst_status_12_24m            0.667770
account_worst_status_3_6m              0.577159
account_worst_status_6_12m             0.603645
age                                    0.000000
avg_payment_span_0_12m                 0.238417
avg_payment_span_0_3m                  0.493168
merchant_category                      0.000000
merchant_group                         0.000000
has_paid                               0.000000
max_paid_inv_0_12m                     0.000000
max_paid_inv_0_24m                     0.000000
name_in_email                          0

Some rows have a very large proportion of nulls.<br>
Depending on the application, we can strategically impute these values. But since I'm not familiar with where these variables are coming from, I'm going to take the convenient route and drop any columns with > 20% null values. We will use <b>XGBoost</b> to automatically handle the remainder of the NULL values.

# Data Cleaning

We need to do a few things here:
<ul>
    <li> Remove the columns with > 20% NULL values </li>
    <li> Convert categorical variables into dummy variables </li>
    <li> Convert Boolean variables from True/False to 1/0 </li>
    <li> Set UUID to the index so that it's not being used as an input variable </li>

### 1. Remove NULLs

In [23]:
null_percentages = raw_dataset.isnull().sum()/raw_dataset.shape[0]
null_valid_indices = [idx for idx in range(null_percentages.shape[0]) if null_percentages.iloc[idx] < 0.2]
print(null_valid_indices)

[0, 1, 2, 3, 4, 5, 12, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]


In [25]:
null_cleaned_dataset = raw_dataset.iloc[:, null_valid_indices]

In [26]:
null_cleaned_dataset.head()

,uuid,default,account_amount_added_12_24m,account_days_in_dc_12_24m,account_days_in_rem_12_24m,account_days_in_term_12_24m,age,merchant_category,merchant_group,has_paid,...,status_2nd_last_archived_0_24m,status_3rd_last_archived_0_24m,status_max_archived_0_6_months,status_max_archived_0_12_months,status_max_archived_0_24_months,recovery_debt,sum_capital_paid_account_0_12m,sum_capital_paid_account_12_24m,sum_paid_inv_0_12m,time_hours
0,63f69b2c-8b1c-4740-b78d-52ed9a4515ac,0.0,0,0.0,0.0,0.0,20,Dietary supplements,Health & Beauty,True,...,1,1,1,1,1,0,0,0,178839,9.653333
1,0e961183-8c15-4470-9a5e-07a1bd207661,0.0,0,0.0,0.0,0.0,50,Books & Magazines,Entertainment,True,...,1,1,1,2,2,0,0,0,49014,13.181389
2,d8edaae6-4368-44e0-941e-8328f203e64e,0.0,0,0.0,0.0,0.0,22,Diversified entertainment,Entertainment,True,...,1,1,1,2,2,0,0,0,124839,11.561944
3,0095dfb6-a886-4e2a-b056-15ef45fdb0ef,0.0,0,NaN,NaN,NaN,36,Diversified entertainment,Entertainment,True,...,1,1,1,1,1,0,0,0,324676,15.751111
4,c8f8b835-5647-4506-bf15-49105d8af30b,0.0,0,0.0,0.0,0.0,25,Electronic equipment & Related accessories,Electronics,True,...,0,0,1,1,1,0,0,0,7100,12.698611


### 2. Dummify Categorical Features

In [27]:
categorical_features = [
    'account_status', 
    'account_worst_status_0_3m',
    'account_worst_status_12_24m',
    'account_worst_status_3_6m',
    'account_worst_status_6_12m',
    'merchant_category',
    'merchant_group',
    'name_in_email',
    'status_last_archived_0_24m',
    'status_2nd_last_archived_0_24m',
    'status_3rd_last_archived_0_24m',
    'status_max_archived_0_6_months',
    'status_max_archived_0_12_months',
    'status_max_archived_0_24_months',
    'worst_status_active_inv'
]
other_features = [val for val in null_cleaned_dataset.columns if val not in categorical_features]

In [29]:
dummy_subset = pd.get_dummies(null_cleaned_dataset.loc[:, categorical_features])
dummy_subset.shape

(99976, 89)

In [31]:
dummified_dataset = pd.concat((null_cleaned_dataset.loc[:, other_features], dummy_subset), axis=1)
dummified_dataset.shape

(99976, 113)

Since we have a large number of columns we can do 2 things to train a better model:
<ol>
    <li> Perform PCA </li>
    <li> Train more and larger XGBoost trees to account for all the variables </li>
</ol><br>
For the MVP's sake, we will simply train more large XGBoost trees instead of using PCA.

### 3. Convert Boolean Features from True/False to 1/0

In [36]:
fixed_boolean_dataset = dummified_dataset.copy()
fixed_boolean_dataset['has_paid'] = fixed_boolean_dataset['has_paid'] * 1
fixed_boolean_dataset['has_paid'].head(2)

0    1
1    1
Name: has_paid, dtype: int64

### 3. Set UUID to the index

In [37]:
dataset = fixed_boolean_dataset.set_index('uuid')
dataset.head()

,default,account_amount_added_12_24m,account_days_in_dc_12_24m,account_days_in_rem_12_24m,account_days_in_term_12_24m,age,has_paid,max_paid_inv_0_12m,max_paid_inv_0_24m,num_active_inv,...,merchant_group_Jewelry & Accessories,"merchant_group_Leisure, Sport & Hobby",name_in_email_F,name_in_email_F+L,name_in_email_F1+L,name_in_email_Initials,name_in_email_L,name_in_email_L1+F,name_in_email_Nick,name_in_email_no_match
uuid,,,,,,,,,,,,,,,,,,,,,
63f69b2c-8b1c-4740-b78d-52ed9a4515ac,0.0,0,0.0,0.0,0.0,20,1,31638.0,31638.0,2,...,0,0,0,0,0,0,0,0,0,1
0e961183-8c15-4470-9a5e-07a1bd207661,0.0,0,0.0,0.0,0.0,50,1,13749.0,13749.0,0,...,0,0,0,1,0,0,0,0,0,0
d8edaae6-4368-44e0-941e-8328f203e64e,0.0,0,0.0,0.0,0.0,22,1,29890.0,29890.0,1,...,0,0,0,0,0,0,0,1,0,0
0095dfb6-a886-4e2a-b056-15ef45fdb0ef,0.0,0,NaN,NaN,NaN,36,1,40040.0,40040.0,1,...,0,0,0,0,1,0,0,0,0,0
c8f8b835-5647-4506-bf15-49105d8af30b,0.0,0,0.0,0.0,0.0,25,1,7100.0,7100.0,0,...,0,0,0,1,0,0,0,0,0,0


Beautiful! Now let's perform out data split and oversampling!

# Train-Validation-Test Split and Oversampling

In [40]:
feature_columns = [val for val in dataset.columns if val != 'default']

In [41]:
holdout_indices = dataset[dataset['default'].isnull()].index.tolist()
X_holdout, y_holdout = dataset.loc[holdout_indices, feature_columns], dataset.loc[holdout_indices, 'default']

In [ ]:
holdout_set = dataset